In [1]:
# just a demo:
# the following redirects future cells through this loop.
# This should make it possible to put wrapper functionality
# around cells to keep the text inside the cells clean.

while sleep 1; do
echo -n "$PS1"
read ln
eval "$ln"
echo "It worked! " # If that last space is removed, it does not work :-(
done


In [1]:
# (1) optional reset of Wakame-vdc
# Comment out the following lines and run cell to remove all
# added images and instances.

# reset-wakame.sh